In [1]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import json
from datetime import datetime 
from alpaca_trade_api.rest import REST, TimeFrame
import time
from time import sleep
import datetime
import csv
%matplotlib inline

In [2]:
# Load .env enviroment variables
load_dotenv()
opensea_api_key = os.getenv("opensea_api_key")
opensea_api_key

'9796bf59fe674cae9688a1a9bb422338'

In [3]:
opensea_collections_url = "https://api.opensea.io/api/v1/collections"
opensea_collection_stats_url = "https://api.opensea.io/api/v1/collection/doodles-official/stats"

In [4]:
def get_events(url,start_date, end_date, event_type, collection_slug,limit=1):
    

    headers = {
        "Accept": "application/json",
        "X-API-KEY": opensea_api_key
    }
    
    query = { 
        "only_opensea": "true", 
        "occurred_before": end_date,
        "occurred_after": start_date,
        "event_type":event_type,
        "collection_slug":collection_slug,
        "limit":limit
    }
    
    response_data = requests.request("GET", url, headers=headers, params=query)
    json_data = response_data.json()
    #print(json_data)
    return json_data

In [5]:
#get_opensea_api_data(opensea_collections_url)
#get_opensea_api_data(opensea_collection_stats_url)

start_date = datetime.datetime(2021, 1, 10)
end_date = datetime.datetime(2022, 4, 11)
event_url = "https://api.opensea.io/api/v1/events";
event_type = "successful"
collection_slug = "doodles-official"
limit =1
response_data = get_events(event_url,
                                     start_date,
                                     end_date,
                                     event_type,
                                     collection_slug,
                                     limit)

#print(response_data)
#print(response_data['asset_events'])


In [6]:
def parse_event(event):
    record = {}
    asset = event.get('asset')
    if asset == None:
        return None # if there's no asset that means it's not a single NFT transaction so skip this item

    #collection
    record['collection_slug'] = asset['collection']['slug']
    record['collection_name'] = asset['collection']['name']
    record['collection_url'] = "https://opensea.io/collection/" + asset['collection']['slug']

    #asset
    record['asset_id'] = asset['id']
    record['asset_name'] = asset['name']
    record['asset_description'] = asset['description']
    record['asset_contract_date'] = asset['asset_contract']['created_date']
    record['asset_url'] = asset['permalink']
    record['asset_img_url'] = asset['image_url']

    #event
    record['event_id'] = event['id']
    record['event_time'] = event.get('created_date')
    record['event_auction_type'] = event.get('auction_type')
    record['event_contract_address'] = event.get('contract_address')
    record['event_quantity'] = event.get('quantity')
    record['event_payment_symbol'] =  None if event.get('payment_token') == None else event.get('payment_token').get('symbol')

    decimals = 18
    if event.get('payment_token') != None:
        decimals = event.get('payment_token').get('decimals')

    price_str = event['total_price']

    try: 
        if len(price_str) < decimals:
            price_str =  "0." + (decimals-len(price_str)) * "0" + price_str
            record['event_total_price'] = float(price_str)
        else:
            record['event_total_price'] = float(price_str[:-decimals] + "." + price_str[len(price_str)-decimals:])
    except:
        print(event)

    return record

In [18]:
pause = 1

def fetch_all_events(url,start_date, end_date,event_type,collection_slug,limit):
    result = list()
    print(f"Fetching events between {start_date} and {end_date}")
    response = get_events(url,start_date,end_date,event_type,collection_slug, limit)
    for event in response['asset_events']:
        cleaned_event = parse_event(event)
        if cleaned_event != None:
            result.append(cleaned_event)
    return result

In [23]:


start_date = datetime.datetime(2021, 2, 10)
end_date = datetime.datetime(2022, 4, 10)
limit = 30

result = fetch_all_events(event_url,start_date,end_date,event_type,collection_slug, limit)

print(f"total number of records found:{len(result)}")

Fetching events between 2021-02-10 00:00:00 and 2022-04-10 00:00:00
total number of records found:30


In [25]:
file_name="event_data.csv"

#print(result[0].keys())

def write_csv(data, file_name):
    with open(file_name, mode='w', encoding='utf-8', newline='\n') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames = data[0].keys())

        writer.writeheader()
        for event in data:
            writer.writerow(event)
            
write_csv(result, file_name)

In [26]:
df = pd.read_csv('event_data.csv')
df

,collection_slug,collection_name,collection_url,asset_id,asset_name,asset_description,asset_contract_date,asset_url,asset_img_url,event_id,event_time,event_auction_type,event_contract_address,event_quantity,event_payment_symbol,event_total_price
0,doodles-official,Doodles,https://opensea.io/collection/doodles-official,74285485,Doodle #5289,A community-driven collectibles project featur...,2021-10-16T16:25:03.736944,https://opensea.io/assets/0x8a90cab2b38dba80c6...,https://lh3.googleusercontent.com/SMCPfXmeX_og...,4857623401,2022-04-09T17:41:24.150762,NaN,0x7f268357a8c2552623316e2562d90e642bb538e5,1,ETH,16.990000
1,doodles-official,Doodles,https://opensea.io/collection/doodles-official,74085204,Doodle #3902,A community-driven collectibles project featur...,2021-10-16T16:25:03.736944,https://opensea.io/assets/0x8a90cab2b38dba80c6...,https://lh3.googleusercontent.com/v43qB-Hjfevs...,4851784397,2022-04-09T13:13:47.041657,NaN,0x7f268357a8c2552623316e2562d90e642bb538e5,1,WETH,14.500000
2,doodles-official,Doodles,https://opensea.io/collection/doodles-official,74062705,Doodle #3209,A community-driven collectibles project featur...,2021-10-16T16:25:03.736944,https://opensea.io/assets/0x8a90cab2b38dba80c6...,https://lh3.googleusercontent.com/OdecB9czyZe5...,4850668460,2022-04-09T12:22:41.507015,NaN,0x7f268357a8c2552623316e2562d90e642bb538e5,1,ETH,16.000000
3,doodles-official,Doodles,https://opensea.io/collection/doodles-official,74041873,Doodle #648,A community-driven collectibles project featur...,2021-10-16T16:25:03.736944,https://opensea.io/assets/0x8a90cab2b38dba80c6...,https://lh3.googleusercontent.com/qEW5MPXCCxqL...,4846619344,2022-04-09T09:25:17.670987,NaN,0x7f268357a8c2552623316e2562d90e642bb538e5,1,ETH,18.290000
4,doodles-official,Doodles,https://opensea.io/collection/doodles-official,74416174,Doodle #9097,A community-driven collectibles project featur...,2021-10-16T16:25:03.736944,https://opensea.io/assets/0x8a90cab2b38dba80c6...,https://lh3.googleusercontent.com/pR6Tj-pLdfYD...,4840739599,2022-04-09T05:00:32.180612,NaN,0x7f268357a8c2552623316e2562d90e642bb538e5,1,ETH,19.000000
5,doodles-official,Doodles,https://opensea.io/collection/doodles-official,74041950,Doodle #675,A community-driven collectibles project featur...,2021-10-16T16:25:03.736944,https://opensea.io/assets/0x8a90cab2b38dba80c6...,https://lh3.googleusercontent.com/PPFfUiM3D0Kh...,4833158694,2022-04-08T22:49:05.801891,NaN,0x7f268357a8c2552623316e2562d90e642bb538e5,1,ETH,15.990000
6,doodles-official,Doodles,https://opensea.io/collection/doodles-official,74042033,Doodle #828,A community-driven collectibles project featur...,2021-10-16T16:25:03.736944,https://opensea.io/assets/0x8a90cab2b38dba80c6...,https://lh3.googleusercontent.com/a6tP0mMnivh_...,4833038880,2022-04-08T22:43:08.167671,NaN,0x7f268357a8c2552623316e2562d90e642bb538e5,1,ETH,16.690000
7,doodles-official,Doodles,https://opensea.io/collection/doodles-official,74411572,Doodle #6022,A community-driven collectibles project featur...,2021-10-16T16:25:03.736944,https://opensea.io/assets/0x8a90cab2b38dba80c6...,https://lh3.googleusercontent.com/fF2dXeiNt_aG...,4831175810,2022-04-08T21:16:00.660267,NaN,0x7f268357a8c2552623316e2562d90e642bb538e5,1,ETH,15.900000
8,doodles-official,Doodles,https://opensea.io/collection/doodles-official,74043174,Doodle #1165,A community-driven collectibles project featur...,2021-10-16T16:25:03.736944,https://opensea.io/assets/0x8a90cab2b38dba80c6...,https://lh3.googleusercontent.com/P1Bd3ZR9AM2m...,4828410378,2022-04-08T19:01:38.586622,NaN,0x7f268357a8c2552623316e2562d90e642bb538e5,1,ETH,15.900000
9,doodles-official,Doodles,https://opensea.io/collection/doodles-official,74106202,Doodle #4184,A community-driven collectibles project featur...,2021-10-16T16:25:03.736944,https://opensea.io/assets/0x8a90cab2b38dba80c6...,https://lh3.googleusercontent.com/r2zdZXqcOlu5...,4826789272,2022-04-08T17:44:50.191902,NaN,0x7f268357a8c2552623316e2562d90e642bb538e5,1,ETH,20.000000
